In [1]:
import sys
sys.path.append('../../')
sys.path.append('../../examples/')

In [2]:
import os
import pickle
import logging

import numpy as np

import torch
import torch.nn as nn
from torch.nn import NLLLoss

from argparse import Namespace
from tqdm import tqdm

from pytorch_pretrained_bert.modeling_fine_tune import BertForPairWiseClassification
from run_classifier_dataset_utils_fine_tune import AtecCcksProcessor, compute_metrics
from run_classifier_dataset_utils_fine_tune import convert_examples_to_features_fine_tune as convert_examples_to_features
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from pytorch_pretrained_bert.tokenization import BertTokenizer

In [3]:
FINE_TUNED_PATH = '/efs/fine_tune/atec_ccks_fine_tune_2/'
task_name = 'atec_ccks'

args = Namespace(data_dir = '/efs/projects/bert_fine_tune/fine_tune/data/train_dev_test/ATEC_CCKS/processed',
                 bert_model = '/efs/downloads/bert/pytorch/bert_base_chinese',
                 max_seq_length = 128,
                 local_rank = -1,
                 eval_batch_size = 8,
                 do_train = False
                )

logger = logging.getLogger("ATEC_CCKS_eval")

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.DEBUG)

In [4]:
device = torch.device('cuda')
processor = AtecCcksProcessor()
tokenizer = BertTokenizer.from_pretrained(FINE_TUNED_PATH)
model = BertForPairWiseClassification.from_pretrained(FINE_TUNED_PATH)
model.to(device)

07/11/2019 07:47:04 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file /efs/fine_tune/atec_ccks_fine_tune_2/vocab.txt
07/11/2019 07:47:04 - INFO - pytorch_pretrained_bert.modeling_fine_tune -   loading weights file /efs/fine_tune/atec_ccks_fine_tune_2/pytorch_model.bin
07/11/2019 07:47:04 - INFO - pytorch_pretrained_bert.modeling_fine_tune -   loading configuration file /efs/fine_tune/atec_ccks_fine_tune_2/config.json
07/11/2019 07:47:04 - INFO - pytorch_pretrained_bert.modeling_fine_tune -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_t

BertForPairWiseClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediat

In [5]:
label_list = processor.get_labels()
num_labels = len(label_list)
eval_examples = processor.get_dev_examples(args.data_dir)

cached_eval_features_file = os.path.join(args.data_dir, 'dev_{0}_{1}_{2}'.format(
            list(filter(None, args.bert_model.split('/'))).pop(),
                        str(args.max_seq_length),
                        str(task_name)))
try:
    with open(cached_eval_features_file, "rb") as reader:
        eval_features = pickle.load(reader)
except:
    eval_features = convert_examples_to_features(
        eval_examples, label_list, args.max_seq_length, tokenizer, output_mode)
    if args.local_rank == -1 or torch.distributed.get_rank() == 0:
        logger.info("  Saving eval features into cached file %s", cached_eval_features_file)
        with open(cached_eval_features_file, "wb") as writer:
            pickle.dump(eval_features, writer)
            
all_input_ids_a = torch.tensor([f.input_ids_a for f in eval_features], dtype=torch.long)
all_input_mask_a = torch.tensor([f.input_mask_a for f in eval_features], dtype=torch.long)
all_segment_ids_a = torch.tensor([f.segment_ids_a for f in eval_features], dtype=torch.long)

all_input_ids_b = torch.tensor([f.input_ids_b for f in eval_features], dtype=torch.long)
all_input_mask_b = torch.tensor([f.input_mask_b for f in eval_features], dtype=torch.long)
all_segment_ids_b = torch.tensor([f.segment_ids_b for f in eval_features], dtype=torch.long)

all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)

eval_data = TensorDataset(all_input_ids_a,
                          all_input_ids_b,
                          all_input_mask_a,
                          all_input_mask_b,
                          all_segment_ids_a,
                          all_segment_ids_b,
                          all_label_ids)

eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=args.eval_batch_size)

model.eval()
eval_loss = 0
nb_eval_steps = 0
preds = []
out_label_ids = None

for (input_ids_a, input_ids_b,
     input_mask_a, input_mask_b,
     segment_ids_a, segment_ids_b,
     label_ids
    ) in tqdm(eval_dataloader, desc="Evaluating"):
    input_ids_a = input_ids_a.to(device)
    input_mask_a = input_mask_a.to(device)
    segment_ids_a = segment_ids_a.to(device)

    input_ids_b = input_ids_b.to(device)
    input_mask_b = input_mask_b.to(device)
    segment_ids_b = segment_ids_b.to(device)

    label_ids = label_ids.to(device)

    with torch.no_grad():
        cos_sim, pos_prob, pout1, pout2 = model(input_ids_1 = input_ids_a,
                                        input_ids_2 = input_ids_b,
                                        token_type_ids_1=segment_ids_a,
                                        token_type_ids_2=segment_ids_b,
                                        attention_mask_1=input_mask_a,
                                        attention_mask_2=input_mask_b,)
        neg_prob = 1 - pos_prob
#         if any(neg_prob) <= 0:
#             logger.debug("invalid neg_prob")
#             print(cos_sim, probs)
#             print(input_ids_a)
#             print(input_ids_b)
#             break
            
        probs = torch.stack([neg_prob, pos_prob], dim = 1)
        log_probs = torch.log(probs)
        
    loss_fct = NLLLoss()
    tmp_eval_loss = loss_fct(log_probs.view(-1, num_labels), label_ids.view(-1))
    if tmp_eval_loss.mean().item() == np.inf:
        logger.debug("invalid loss")
        print(cos_sim, probs)
        print(input_ids_a)
        print(input_ids_b)
        break
    
    eval_loss += tmp_eval_loss.mean().item()
    nb_eval_steps += 1
    if len(preds) == 0:
        preds.append(probs.detach().cpu().numpy())
        out_label_ids = label_ids.detach().cpu().numpy()
    else:
        preds[0] = np.append(
            preds[0], probs.detach().cpu().numpy(), axis=0)
        out_label_ids = np.append(
            out_label_ids, label_ids.detach().cpu().numpy(), axis=0)

eval_loss = eval_loss / nb_eval_steps

Evaluating: 100%|██████████| 1250/1250 [02:49<00:00,  7.16it/s]


In [6]:
preds = preds[0]
preds = np.argmax(preds, axis=1)
result = compute_metrics(task_name, preds, out_label_ids)

loss = tr_loss/global_step if args.do_train else None

result['eval_loss'] = eval_loss
# result['global_step'] = global_step
result['loss'] = loss

# output_eval_file = os.path.join(args.output_dir, "eval_results.txt")
logger.info("***** Eval results *****")
for key in sorted(result.keys()):
    logger.info("  %s = %s", key, str(result[key]))
#         writer.write("%s = %s\n" % (key, str(result[key])))

07/11/2019 07:50:04 - INFO - ATEC_CCKS_eval -   ***** Eval results *****
07/11/2019 07:50:04 - INFO - ATEC_CCKS_eval -     acc = 0.5666
07/11/2019 07:50:04 - INFO - ATEC_CCKS_eval -     acc_and_f1 = 0.5670325349301397
07/11/2019 07:50:04 - INFO - ATEC_CCKS_eval -     eval_loss = 0.6656205968379975
07/11/2019 07:50:04 - INFO - ATEC_CCKS_eval -     f1 = 0.5674650698602794
07/11/2019 07:50:04 - INFO - ATEC_CCKS_eval -     loss = None


In [7]:
eval_examples[193].text_a, eval_examples[193].text_b

('帮查一下支付宝还款花呗明细', '如何查看花呗分期后几个月还款明细')

In [8]:
eval_examples[194].text_a, eval_examples[194].text_b

('花呗自动还款会成功扣去工商银行储蓄卡上的钱吗', '如果银卡有钱花呗会自动还款吗')

In [9]:
eval_examples[195].text_a, eval_examples[195].text_b

('为什么我没有微粒贷 为什么我没有微贷粒', '为什么我没有呢')

In [10]:
eval_data[195]

(tensor([ 101,  711,  784,  720, 2769, 3766, 3300, 2544, 5108, 6587,  711,  784,
          720, 2769, 3766, 3300, 2544, 6587, 5108,  102,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]),
 tensor([ 101,  711,  784,  720, 2769, 3766, 3300, 1450,  102,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,

In [11]:
a = torch.tensor(1.0)

In [12]:
torch.clamp(a, min = 0, max = 0.99999)

tensor(1.0000)

In [13]:
torch.clamp?

In [14]:
a.dtype

torch.float32